# Maps of RMSSSH

In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()
c

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:46127 Dashboard: http://127.0.0.1:33989/status,Cluster Workers: 8 Cores: 48 Memory: 118.00 GB


## 0. Modules and libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")
import os 
import time 
import glob


import numpy as np
import xarray as xr
import time

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

%matplotlib inline


## 1. Data 

### eNATL60

In [4]:
ssh_std_eNATL60=xr.open_dataset('/store/albert7a/eNATL60/eNATL60-BLBT02-S/1h/SSH-MEAN-STD/eNATL60-BLBT02_std-ssh_1d.nc',chunks={'x':500,'y':500})
ssh_mean_eNATL60=xr.open_dataset('/store/albert7a/eNATL60/eNATL60-BLBT02-S/1h/SSH-MEAN-STD/eNATL60-BLBT02_mean-ssh_1d.nc',chunks={'x':500,'y':500})
grid_eNATL6O=xr.open_dataset('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mask_eNATL60_3.6_lev1.nc',chunks={'x':500,'y':500})

### AVISO

In [6]:
ssh_std_AVISO=xr.open_dataset('/scratch/cnt0024/hmg2840/albert7a/AVISO/AVISO_y2009m06d30-y2010m10d29_std-ssh.nc',chunks={'latitude':500,'longitude':500})
ssh_mean_AVISO=xr.open_dataset('/scratch/cnt0024/hmg2840/albert7a/AVISO/AVISO_y2009m06d30-y2010m10d29_mean-ssh.nc',chunks={'latitude':500,'longitude':500})
grid_AVISO=xr.open_dataset('/scratch/cnt0024/hmg2840/albert7a/AVISO/dt_global_allsat_phy_l4_20101225_20190101.nc',chunks={'latitude':500,'longitude':500})

## 2. Some functions

In [7]:
# normalization
def normalize(data=None,lat=None,lon=None): # not clean : not area weighted
    # data is a masked array, 
    # lat, lon can be xr.dataarray
    alon = np.mod( lon + 180,360) - 180
    alat = np.array(lat)
    condition =     (alat >27) * (alat <65) * (alon >-80) * (alon < 3)
    refvalue = np.mean(data[np.where(condition)])
    return data - refvalue

## 3. Maps of RMSSSH

In [8]:
fig=plt.figure(figsize=(20,25))
ax = fig.add_subplot(211,projection=ccrs.PlateCarree(central_longitude=-30))
ax.set_extent([-100, 50, 0, 70])

data = ssh_std_AVISO.std_sossheig
lat = grid_AVISO.latitude
lon = grid_AVISO.longitude

pcolor=ax.pcolormesh(lon,lat,data,transform=ccrs.PlateCarree(),cmap=mplcm.YlGnBu,vmin=0,vmax=1)
ax.add_feature(cfeature.LAND,facecolor='grey')
ax.coastlines()
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

fig.subplots_adjust(right=0.8)
gl.xlocator = mticker.FixedLocator([-110,-90,-70,-50,-30,-10,10,30,50,70,90,110])
gl.ylocator = mticker.FixedLocator([-10,0,10,20,30,40,50,60,70,80])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.xlabels_top = False
gl.ylabels_left = False

cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m')
ax.set_title('AVISO',size=17,y=1.08)


ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=-30))
ax.set_extent([-100, 50, 0, 70])

data = ssh_std_eNATL60.std_sossheig
lat = grid_eNATL6O.nav_lat
lon = grid_eNATL6O.nav_lon

pcolor=ax.pcolormesh(lon,lat,data,transform=ccrs.PlateCarree(),cmap=mplcm.YlGnBu,vmin=0,vmax=1)
ax.add_feature(cfeature.LAND,facecolor='grey')
ax.coastlines()
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

fig.subplots_adjust(right=0.8)
gl.xlocator = mticker.FixedLocator([-110,-90,-70,-50,-30,-10,10,30,50,70,90,110])
gl.ylocator = mticker.FixedLocator([-10,0,10,20,30,40,50,60,70,80])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.xlabels_top = False
gl.ylabels_left = False

cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m')
ax.set_title('eNATL60',size=17,y=1.08)
 

Text(0.5, 1.08, 'eNATL60')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x2ac755302e18> (for post_execute):


URLError: <urlopen error [Errno 110] Connection timed out>

URLError: <urlopen error [Errno 110] Connection timed out>

<Figure size 1440x1800 with 4 Axes>

In [23]:
fig=plt.figure(figsize=(20,25))
ax = fig.add_subplot(211,projection=ccrs.PlateCarree(central_longitude=-30))
ax.set_extent([-100, 50, 0, 70])

data = ssh_mean_AVISO.mean_sossheig
lat = grid_AVISO.latitude
lon = grid_AVISO.longitude

pcolor=ax.pcolormesh(lon,lat,data,transform=ccrs.PlateCarree(),cmap=mplcm.YlGnBu,vmin=0,vmax=0.5)
ax.add_feature(cfeature.LAND,facecolor='grey')
ax.coastlines()
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

fig.subplots_adjust(right=0.8)
gl.xlocator = mticker.FixedLocator([-110,-90,-70,-50,-30,-10,10,30,50,70,90,110])
gl.ylocator = mticker.FixedLocator([-10,0,10,20,30,40,50,60,70,80])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.xlabels_top = False
gl.ylabels_left = False

cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m')
ax.set_title('AVISO',size=17,y=1.08)


ax = fig.add_subplot(212,projection=ccrs.PlateCarree(central_longitude=-30))
ax.set_extent([-100, 50, 0, 70])

data = ssh_mean_eNATL60.mean_sossheig
lat = grid_eNATL6O.nav_lat
lon = grid_eNATL6O.nav_lon

pcolor=ax.pcolormesh(lon,lat,data,transform=ccrs.PlateCarree(),cmap=mplcm.YlGnBu,vmin=0,vmax=0.5)
ax.add_feature(cfeature.LAND,facecolor='grey')
ax.coastlines()
gl=ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

fig.subplots_adjust(right=0.8)
gl.xlocator = mticker.FixedLocator([-110,-90,-70,-50,-30,-10,10,30,50,70,90,110])
gl.ylocator = mticker.FixedLocator([-10,0,10,20,30,40,50,60,70,80])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.xlabels_top = False
gl.ylabels_left = False

cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m')
ax.set_title('eNATL60',size=17,y=1.08)
 

Text(0.5, 1.08, 'eNATL60')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x2b95eee0c0d0> (for post_execute):


URLError: <urlopen error [Errno 110] Connection timed out>

URLError: <urlopen error [Errno 110] Connection timed out>

<Figure size 1440x1800 with 4 Axes>